<a href="https://colab.research.google.com/github/Gyerzix/test/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext sql
%sql sqlite:///base.db

'Connected: @base.db'

### Написание функций для работы с sqlite с использованием python

In [ ]:
%%sql
Pragma foreign_keys=on;

DROP TABLE if exists Invoice_details;
DROP TABLE if exists Products;
DROP TABLE if exists Invoices;
DROP TABLE if exists Customers;

CREATE TABLE Customers (
          cust_ID integer NOT NULL Primary key, 
          cust_name varchar, 
          cust_info varchar
          ); 

CREATE TABLE Invoices (
          invoice_ID integer NOT NULL Primary key,
          customer_id varchar NOT NULL,
          order_sum integer,
          invoice_date integer NOT NULL,
          FOREIGN KEY (customer_id) REFERENCES Customers(cust_ID));

CREATE TABLE Products (
          product_id integer NOT NULL Primary key,
          product_name varchar NOT NULL, 
          price integer);

CREATE TABLE Invoice_details (
          invoice_ID integer NOT NULL, 
          product_id integer NOT NULL,
          qty integer NOT NULL,
          primary key (product_id, invoice_id)
          FOREIGN KEY (product_id) REFERENCES Products(product_id)
          FOREIGN KEY (invoice_ID) REFERENCES Invoices(invoice_ID));

 * sqlite:///base.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [ ]:
import sqlite3
db = sqlite3.connect('base.db')

In [ ]:
type(db)

sqlite3.Connection

In [ ]:
cur = db.cursor() # cursor(factory=Cursor) - метод для вызова cursor - возвращает объект типа сursor

In [ ]:
cur.execute("""SELECT * 
            FROM Customers""") # execute(sql[, parameters]) - создание промежуточного объекта типа cursor и вызов метода execute объекта cursor

In [ ]:
a = cur.fetchone() # вывод 1 строки

In [ ]:
a

In [ ]:
cur.fetchall() # вывод всего

[(1, 'Microsoft', 'computer software'),
 (2, 'Google', 'internet-related services')]

In [ ]:
%sql SELECT * FROM Customers

 * sqlite:///base.db
Done.


cust_ID,cust_name,cust_info
1,Microsoft,computer software
2,Google,internet-related services
3,Microsoft,computer software
4,Google,internet-related services


Написать функцию для добавления клиента (имя, инфо):

In [ ]:
def insert_customer(name, info, db):
    cur = db.cursor()
    cur.execute('''
    SELECT COALESCE(MAX(cust_ID)+1, 1) FROM Customers''')
    ID = cur.fetchone()[0]
    cur.execute('''
        INSERT INTO Customers(cust_ID, cust_name, cust_info) 
        VALUES(?,?,?)''', (ID,name,info))
    db.commit() # сохранить изменения

In [ ]:
insert_customer('Microsoft', 'computer software', db)
insert_customer('Google', 'internet-related services', db)

In [ ]:
%%sql
select * from customers;

 * sqlite:///base.db
Done.


cust_ID,cust_name,cust_info
1,Microsoft,computer software
2,Google,internet-related services
3,Microsoft,computer software
4,Google,internet-related services


Написать функцию для добавления продукта (имя, цена):

In [ ]:
def insert_products(name,price):
    cur=db.cursor()
    cur.execute('select coalesce(max(product_id)+1,1) from products')
    id=cur.fetchone()[0]
    cur.execute('insert into products(product_id, product_name,price) values(?,?,?)', (id,name,price))
    db.commit()

In [ ]:
insert_products('Win10', 25000)
insert_products('Google glass', 2000)

In [ ]:
%%sql 
select * from products;

 * sqlite:///base.db
Done.


product_id,product_name,price
1,Win10,25000
2,Google glass,2000


Написать функцию для добавления заказа. 

Вход: параметр_1 - имя клиента, 
параметр_2 - список продуктов вида [['a',1],['b',2]]

1 шаг - проверка, что такие продукты и клиенты существуют, 
2 шаг - добавление в таблицы invoices и invoice_details

In [ ]:
def check_client(name): # Проверяем, есть ли такой клиент 
    cur = db.cursor()
    cur.execute('''select cust_id from customers where cust_name = ?''', (name,))
    try:
        cli_id = cur.fetchone()[0]
        return cli_id
    except TypeError:
        print('Такого клиента не существует')
        return -1
    except:
        print('Такого клиента не существует')
        return 0

In [ ]:
def check_products(product_list): # Проверяем, есть ли такой продукт
    cur = db.cursor()
    res = []
    for a, qty in product_list:
        cur.execute('''select product_id from products where product_name = ? ''', (a, ))
        try: 
            pr_id = cur.fetchone()[0]
            res.append([pr_id, qty])
        except TypeError:
            print('Такой продукт не существует')
            return -1 
    return res

In [ ]:
def add_invoice(customer_id, invoice_date='2021-11-24'):
    cur = db.cursor()
    cur.execute('''select coalesce(max(invoice_id) + 1, 1) from invoices''')
    inv_id = cur.fetchone()[0]
    cur.execute('''insert into invoices(invoice_id, customer_id, order_sum, invoice_date)
                    VALUES(?,?,NULL,?)''', (inv_id, customer_id, invoice_date))
    db.commit()
    return inv_id

In [ ]:
def add_products(product_list, inv_id):
    cur = db.cursor()
    for name, qty in product_list:
        cur.execute('''insert into invoice_details(invoice_id, product_id, qty) 
                        values (?,?,?)''', (inv_id, name, qty))
    db.commit()
    return 1

In [ ]:
def update_invoice(inv_id):
    cur = db.cursor()
    cur.execute('''
    select sum(ii.qty*price)
        from invoice_details ii
        inner join products p
        on ii.product_id = p.product_id 
        where invoice_id = ?''', (str(inv_id)))
    sum_order = cur.fetchone()[0]
    cur.execute('''Update invoices set order_sum = ? where invoice_id = ?''', (sum_order, inv_id))
    db.commit()

In [ ]:
def add_order(client_name, product_list):
    client_id = check_client(client_name)
    if client_id==-1:
        return -1 
    prod_list = check_products(product_list)
    if prod_list == -1:
        return -2
    inv_id = add_invoice(client_id)
    add_products(prod_list, inv_id)
    update_invoice(inv_id)
    db.commit()

In [ ]:
add_order('Microsoft', [['Win10',5]])

In [ ]:
%%sql 
SELECT * 
from invoices

 * sqlite:///base.db
Done.


invoice_ID,customer_id,order_sum,invoice_date
1,1,125000,2021-11-24


In [ ]:
%%sql
select * 
from invoice_Details

 * sqlite:///base.db
Done.


invoice_ID,product_id,qty
1,1,5
